In [2]:
# Preparations
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import Latex
import warnings
from PrettyTable import PrettyTable
warnings.filterwarnings("ignore", module="matplotlib")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['font.family'] = "STIX"
plt.rcParams['text.latex.preamble'] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

sns.set(color_codes=True)

def average(data):
    return 1 / len(data) * sum(data)

def error(data, average_of_data):
    s = sum([(x - average_of_data)**2 for x in data])
    return math.sqrt(s / (len(data) * (len(data) - 1)))

def std_deviation(error_of_average, length_of_dataset):
    return error_of_average * math.sqrt(length_of_dataset)

def average_with_weights(data, weights):
    d = data
    w = weights
    return (d * w**-2).sum() / (w**-2).sum()

def error_with_weights(weights):
    w = weights
    return 1 / math.sqrt((w**-2).sum())

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w**-2).sum() / (w**-2).sum()

def werr(group, weight_name):
    return 1 / math.sqrt((group[weight_name]**-2).sum())

# Arbeitsgrundlagen

Im Versuch M1 geht es darum die Geschwindigkeit einer Pistolenkugel auf verschiedene Arten zu bestimmen. Dabei gibt es noch verschiedene Arten von Geschwindigkeiten.

## Flugzeitmethode

Die __mittlere Geschwindigkeit__ eines Objektes kann durch \ref{eq:velocity} berechnet werden.

\begin{equation}
\overline{v} = \frac{s}{t}
\label{eq:velocity}
\end{equation}

Wenn also die Abschusszeit und die Aufprallzeit, sowie die Flugstrecke bekannt sind, so kann die Zeitdifferenz und somit die mittlere Fluggeschwindigkeit des Geschosses ermittelt werden.

## Ballistische Methode

Der Impuls p ist gegeben durch die Gleichung in \ref{eq:impulse}.  Dieser Impuls kann beim Aufprall der Pistolenkugel auf ein ballistisches Pendel gemessen werden. Durch den erheblichen Massenunterschied der Kugel und des Pendels bleibt die Kugel stecken. Ein inelastischer Stoss resultiert. Aus diesem Grund wird die komplette kinetische Energie in Wärme umgewandelt. Der Energieerhaltungssatz ist hier also schwierig anwendbar.

\begin{equation}
p = mv = (m + M)\cdot u
\label{eq:impulse}
\end{equation}

Die verbleibende kinetische Energie wird beim Ausschlagen des Pendels in potentielle Hubenergie umgewandelt.
Mithilfe des Energiesatzes wird die Geschwindigkeit u durch die Hubhöhe h mit Gleichung \ref{eq:hubheight} ersetzt.

\begin{equation}
u = \sqrt{2gh}
\label{eq:hubheight}
\end{equation}

![Ballistische Methode](images/pendulum.png)

Aus Grafik \ref{fig:Ballistische Methode} können die Gleichungen \ref{eq:h} und \ref{eq:cosphi} bestimmt werden.

\begin{equation}
h = l \cdot (1 - cos\varphi)
\label{eq:h}
\end{equation}

\begin{equation}
\cos\varphi = \frac{a}{\sqrt{a^2 + x^2}}
\label{eq:cosphi}
\end{equation}

Da der Schwerpunkt und somit die Länge l des Pendels nur ungenau ermittelt werden können, wird ein Umweg über die Schwingungsdauer T ersichtlich in \ref{eq:Schwingungsdauer} gewählt.

\begin{equation}
T = 2\pi\cdot\sqrt{\frac{l}{g}}
\label{eq:Schwingungsdauer}
\end{equation}

Daraus resultiert die Gleichung \ref{eq:u}.

\begin{equation}
u = \frac{g}{2\pi}\cdot T\cdot\sqrt{2\cdot\Bigg(1-\frac{1}{\sqrt{1 + (\frac{x}{a})^2}}\Bigg)}
\label{eq:u}
\end{equation}

Mit je einer Taylorentwicklung kann die Gleichung \label{eq:u} auf Gleichung \label{eq:u_taylor} angenähert werden.

\begin{equation}
u = \frac{g}{2\pi}\cdot\frac{M+m}{m}\cdot T\cdot\frac{x}{a}\cdot\Bigg[1-\frac{3}{8}\cdot\Big(\frac{x}{a}\Big)^2\Bigg]
\label{eq:u_taylor}
\end{equation}

Da die Taylorreihe nur bis zum Grad 2 entwickelt wurde, wäre der nächste Term der Grässenordnung $10^{-4}$. Es ergibt sich also ein sehr genauer Wert.

Somit sind während des Experimentes die Grössen m, M, a, x und T zu bestimmen.

## Drehstossmethode

Bei dieser Methode wird ein Geschoss wie in Abbildung \ref{fig:Drehstossmethode} gezeigt ein Geschoss auf eine um eine starre Achse frei drehbar gelagerte Hantel abgeschossen.

![Drehstossmethode](images/drehstoss.png)

Für eine mit v geradlinig bewegte Punktmasse m kann der Drehimpuls bezüglich der Drehachse a mit Gleichung \ref{eq:drehimpuls} bestimmt werden.

\begin{equation}
\vec{L_a}=\vec{r}\times\vec{p}=m\cdot (\vec{r}\times\vec{v})
\label{eq:drehimpuls}
\end{equation}

Ferner ist bekannt, dass wie in Gleichung \ref{eq:gesamtdrehimpuls} beschrieben, der Gesamtdrehimpuls des Systems erhalten bleibt.

\begin{equation}
L_a=L_{a,Kugel}+L_{a,Hantel}=const.
\label{eq:gesamtdrehimpuls}
\end{equation}

Daraus folgt, dass für die anfänglich ruhende Hantel Gleichung \ref{eq:ruhe} gilt welche dann gleichzusetzen ist mit der Gleichung \ref{eq:bewegung} der bewegten Hantel nach dem Aufprall.

\begin{equation}
L_a=m\cdot d\cdot v+0
\label{eq:ruhe}
\end{equation}

\begin{equation}
L_a=\omega\cdot(I_{a, H}+m\cdot d^2)
\label{eq:bewegung}
\end{equation}

Daraus abgeleitet ergibt sich für die Geschwindigkeit der Pistolenkugel beim Aufprall die Gleichung \ref{eq:vdrehstoss}, wobei $I_{a, H}$ dem Inneren Moment der Hantel bezüglich a und $\omega$ der Winkelgeschwingikeit der Hantel um die Achse a entspricht.

\begin{equation}
v=\omega\cdot\bigg(\frac{I_{a, H}}{m\cdot d}+d\bigg)
\label{eq:vdrehstoss}
\end{equation}

Somit sind für dieses Experiment die Grössen $I_{a, H}$, m, d und $\omega$ zu bestimmen.

## Einfluss des Luftwiderstandes auf die Geschossgeschwindigkeit

Bisher wurde gezeigt wie die mittlere, sowie die Aufprallgeschwindigkeit ermittelt werden können. Interessant ist aber auch die Mündungsgeschwindigkeit des Geschosses.

Diese kann man sehr gut über den wirkenden Luftwiderstand bestimmen. Der Luftwiderstand ist gegeben durch die Beziehung in \ref{eq:luftwiderstand}, wobei v die Momentangeschwidigkeit des Geschosses, A die Querschnittfläche jenes Geschosses, $\rho_L$ die Luftdichte und c_w der Widerstandsbeiwert des Projektils sind.

\begin{equation}
F_L=\frac{1}{2}c_\omega\cdot A\cdot\rho_L\cdot v^2
\label{eq:luftwiderstand}
\end{equation}

Das Projektil verliert ebensoviel Kinetische Energie wie der Luftwiderstand Bremsarbeit verrichtet. Dies is mit Gleichungen \ref{eq:kinetische_energie} und \ref{eq:kinetische_energie_eingesetzt} ersichtlich.

\begin{equation}
dE_{kin}=-F_Lds
\label{eq:kinetische_energie}
\end{equation}

\begin{equation}
m\cdot v\cdot dv=-\frac{1}{2}c_\omega\cdot A\cdot\rho_L\cdot v^2\cdot ds
\label{eq:kinetische_energie_eingesetzt}
\end{equation}

Daraus kann die Differenzengleichung \ref{eq:kin_diff} gebildet werden. Mit den Anfangsbedingungen s = 0 und v = $v_0$ wird Gleichung \ref{eq:v0} erhalten. Diese kann für sehr kleine $k\cdot s$ mit \ref{eq:v0_approx} angenähert werden.

\begin{equation}
\frac{dv}{v}=-k\cdot ds
\label{eq:kin_diff}
\end{equation}

\begin{equation}
v=v_0\cdot e^{-k\cdot s}
\label{eq:v0}
\end{equation}

\begin{equation}
v=v_0\cdot (1-k\cdot s)
\label{eq:v0_approx}
\end{equation}

k ist in allen diesen Gleichungen mit dem Term $k = \frac{c_\omega\cdot A\cdot\rho_L}{2m}$ zu beziffern.

# Versuchsaufbau

# Versuchsdurchführung

In [4]:
v1_s = 0
v1_s_u = 0
v1_m = 0
v1_m_u = 0


## Ballistische Methode

In [5]:
%%file data/ballistsisch_1.csv
measurement,x

Writing data/ballistsisch_1.csv


In [ ]:
# Evaluate Data

# Read Data
v1_df = pd.read_csv('data/ballistisch.csv')
v1_M = 0
v1_M_u = 0
v1_a = 0
v1_a_u = 0

# Calculate mean etc.
v1_mean = v1_df.mean()['t']
v1_sem = v1_df.sem()['t']
v1_std = v1_df.std()['t']

## Drehstossmethode

In [3]:
# Evaluate Data

# Read Data
v1_df = pd.read_csv('data/drehstoss.csv')
v1_inertia = 0
v1_inertia_u = 0
v1_d = 0
v1_d_u = 0

# Calculate mean etc.
v1_mean = v1_df.mean()['t']
v1_sem = v1_df.sem()['t']
v1_std = v1_df.std()['t']

KeyError: 't'